In [2]:
import langdetect
import sys
import json
from collections import defaultdict
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
pd.set_option('display.max_colwidth', None)
import stanza
# stanza.download('uk')
nlp = stanza.Pipeline(lang='uk')
from nltk.tokenize import word_tokenize

2020-04-22 13:57:06 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-04-22 13:57:06 INFO: Use device: gpu
2020-04-22 13:57:06 INFO: Loading: tokenize
2020-04-22 13:57:06 INFO: Loading: mwt
2020-04-22 13:57:06 INFO: Loading: pos
2020-04-22 13:57:07 INFO: Loading: lemma
2020-04-22 13:57:07 INFO: Loading: depparse
2020-04-22 13:57:08 INFO: Done loading processors!


In [3]:
# !cat jsons/*.json | jq -c '.data | .comments[] | .commentdatajson | {mark,text,dignity,shortcomings}' | jq -s > electronics.json

In [4]:
with open('electronics.json') as f:
    content = json.load(f)

In [5]:
df = pd.DataFrame(data=content)
df = df[~df['mark'].isnull()]
df = df[~((df['dignity']=='')&(df['shortcomings']=='')&(df['text'].str.contains('?', regex=False)))]
df.head()

,mark,text,dignity,shortcomings
0,5,"Апарат брав не тут. Смарт супер. Екран насичений, великий. Телефон зручно лежить в руці, принаймні в мене. Зелений корпус дивиться круто! Швидкий. Батарея тримає день. В силіконовому чохлі не слизький. Камера норм. Селфі камера - так і є нахил в право. Трішки нервує, але не критично. В комлекті були навушники та заводська плівка. З мінусів це датчик приблеження. В телефоній розмові норм, але вайбер, месенджер просто жах. По роботі багато користуюсь цим софтом. Відключаєш вухом все!!! + немає індикатора повідомлень.","Батарея, екран, вигляд.","Датчик приближення, нахил селфі камери."
1,5,"За цю вартість відмінний смартфон. Недоліків на даний час не виявлено. Цікаве рішення з фронтальною камерою. Аккумулятора вистачає. Купувався апарат на подарунок, але налаштовувався мною. Тому пишу лише перші враження.","Ціна, екран, сенсор",На даний момент ще не виявлено
2,5,Класний телефон з цікавим дизайном. Батарея тримає 2 дні. Гарний екран.,,Важко знайти чохол.
4,5,Телефон отримала буквально на наступний день. Літаю з ним пару днів і поки що ( надіюся і надалі) нічого поганого сказати не можу.,,
5,5,"Класний, шикарний, просто бомба телефон за свої Гроші. Після включення зразу прилетіла обнова.","Екран, камера, батарея,сканер відбитку пальця, телефон суцільна перевага.",Не виявлено


In [6]:
import re

def clean(text):
    text = text.replace('\n', '')
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text.lower())

In [7]:
df['text'] = df['text'].apply(clean)
df = df[df['text']!='']
df['mark'].value_counts()

5    6104
4    1580
1     637
3     588
2     403
Name: mark, dtype: int64

In [8]:
df.head()

,mark,text,dignity,shortcomings
0,5,"апарат брав не тут. смарт супер. екран насичений, великий. телефон зручно лежить в руці, принаймні в мене. зелений корпус дивиться круто! швидкий. батарея тримає день. в силіконовому чохлі не слизький. камера норм. селфі камера - так і є нахил в право. трішки нервує, але не критично. в комлекті були навушники та заводська плівка. з мінусів це датчик приблеження. в телефоній розмові норм, але вайбер, месенджер просто жах. по роботі багато користуюсь цим софтом. відключаєш вухом все!!! + немає індикатора повідомлень.","Батарея, екран, вигляд.","Датчик приближення, нахил селфі камери."
1,5,"за цю вартість відмінний смартфон. недоліків на даний час не виявлено. цікаве рішення з фронтальною камерою. аккумулятора вистачає. купувався апарат на подарунок, але налаштовувався мною. тому пишу лише перші враження.","Ціна, екран, сенсор",На даний момент ще не виявлено
2,5,класний телефон з цікавим дизайном. батарея тримає 2 дні. гарний екран.,,Важко знайти чохол.
4,5,телефон отримала буквально на наступний день. літаю з ним пару днів і поки що ( надіюся і надалі) нічого поганого сказати не можу.,,
5,5,"класний, шикарний, просто бомба телефон за свої гроші. після включення зразу прилетіла обнова.","Екран, камера, батарея,сканер відбитку пальця, телефон суцільна перевага.",Не виявлено


In [9]:
df['target'] = df['mark'].apply({5.0: 'pos', 4.0: 'pos', 3.0: 'neg', 2.0: 'neg', 1.0: 'neg'}.get)
df['target'].value_counts(normalize=True)

pos    0.825172
neg    0.174828
Name: target, dtype: float64

# Baseline : MultinomialNB

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.3, random_state=42)

In [11]:
vectorizer = CountVectorizer()
vec = vectorizer.fit(X_train.append(X_test))
print("Total number of features: ", len(vec.get_feature_names()))

Total number of features:  28115


In [12]:
train_features_vectorized = vec.transform(X_train)
test_features_vectorized = vec.transform(X_test)

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_features_vectorized, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
predicted = clf.predict(test_features_vectorized)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

         neg       0.68      0.37      0.48       515
         pos       0.87      0.96      0.91      2279

    accuracy                           0.85      2794
   macro avg       0.78      0.67      0.70      2794
weighted avg       0.84      0.85      0.83      2794



# LogisticRegression + lemma + bigrams

In [15]:
def lemm(x):
    doc = nlp(x)
    lemmas = []
    for sent in doc.sentences:
        for token in sent.words:
            lemmas.append(token.lemma)
    res = ' '.join(l for l in lemmas if l)
    return res

In [16]:
%%time
df['lemma_text'] = df['text'].apply(lemm)
# df.to_csv('lemma.csv')

CPU times: user 12min, sys: 970 ms, total: 12min 1s
Wall time: 12min 1s


In [17]:
# df = pd.read_csv('lemma.csv')
df.head()

,mark,text,dignity,shortcomings,target,lemma_text
0,5,"апарат брав не тут. смарт супер. екран насичений, великий. телефон зручно лежить в руці, принаймні в мене. зелений корпус дивиться круто! швидкий. батарея тримає день. в силіконовому чохлі не слизький. камера норм. селфі камера - так і є нахил в право. трішки нервує, але не критично. в комлекті були навушники та заводська плівка. з мінусів це датчик приблеження. в телефоній розмові норм, але вайбер, месенджер просто жах. по роботі багато користуюсь цим софтом. відключаєш вухом все!!! + немає індикатора повідомлень.","Батарея, екран, вигляд.","Датчик приближення, нахил селфі камери.",pos,"апарат брати не тут . смарт супер . екран насичений , великий . телефон зручно лежати в рука , принаймні в я . зелений корпус дивитися круто ! швидкий . батарея тримати день . в силіконовий чохля не слизький . камера норма . селфі камера - так і бути нахил в право . трішки нервувати , але не критично . в комлект бути навушник та заводський плівка . з мінус це датчик приблеження . в телефонія розмова норма , але вайбер , месенджер просто жах . по робота багато користуватися цей софт . відключати вухо все !!! + немати індикатор повідомлення ."
1,5,"за цю вартість відмінний смартфон. недоліків на даний час не виявлено. цікаве рішення з фронтальною камерою. аккумулятора вистачає. купувався апарат на подарунок, але налаштовувався мною. тому пишу лише перші враження.","Ціна, екран, сенсор",На даний момент ще не виявлено,pos,"за цей вартість відмінний смартфон . недолік на даний час не виявити . цікавий рішення з фронтальний камера . аккумулятора вистачати . купуватися апарат на подарунок , але налаштовуватися я . тому писати лише перший враження ."
2,5,класний телефон з цікавим дизайном. батарея тримає 2 дні. гарний екран.,,Важко знайти чохол.,pos,класний телефон з цікавий дизайн . батарея тримати 2 день . гарний екран .
4,5,телефон отримала буквально на наступний день. літаю з ним пару днів і поки що ( надіюся і надалі) нічого поганого сказати не можу.,,,pos,телефон отримати буквально на наступний день . літати з він пара день і поки що ( надіятися і надалі ) ніщо поган сказати не могти .
5,5,"класний, шикарний, просто бомба телефон за свої гроші. після включення зразу прилетіла обнова.","Екран, камера, батарея,сканер відбитку пальця, телефон суцільна перевага.",Не виявлено,pos,"класний , шикарний , просто бомба телефон за свій гроші . після включення зразу прилетіти обнова ."


In [19]:
X_train, X_test, y_train, y_test = train_test_split(df['lemma_text'], df['target'], test_size=0.3, random_state=42)

In [20]:
vectorizer = CountVectorizer(ngram_range = (1,2))
vec = vectorizer.fit(X_train.append(X_test))
print("Total number of features: ", len(vec.get_feature_names()))

Total number of features:  144277


In [21]:
train_features_vectorized = vec.transform(X_train)
test_features_vectorized = vec.transform(X_test)

In [22]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial",
                         max_iter=1000, verbose=1)
lrc.fit(train_features_vectorized, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 7 seconds


/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.5s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=1,
                   warm_start=False)

In [23]:
# prediction on bigram words
predicted = lrc.predict(test_features_vectorized)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

         neg       0.76      0.50      0.60       515
         pos       0.90      0.96      0.93      2279

    accuracy                           0.88      2794
   macro avg       0.83      0.73      0.77      2794
weighted avg       0.87      0.88      0.87      2794



In [24]:
from sklearn.model_selection import cross_val_score
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial",
                         max_iter=1000, verbose=1)
scores = cross_val_score(lrc, train_features_vectorized, y_train, cv=5, scoring='f1_macro')
print(scores)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 6 seconds


/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 7 seconds


/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 6 seconds


/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 6 seconds
[0.72791149 0.7765491  0.76364494 0.73471981 0.73471981]


/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s finished


In [25]:
print(f'Cross validation score = {scores.mean()}')

Cross validation score = 0.7475090311750577


# Deal with imbalanced dataset

In [26]:
df['target'].value_counts()

pos    7684
neg    1628
Name: target, dtype: int64

In [27]:
# Shuffle the Dataset.
shuffled_df = df.sample(frac=1,random_state=4)

# Put all the fraud class in a separate dataset.
neg_df = shuffled_df.loc[shuffled_df['target'] == 'neg']

#Randomly select 492 observations from the non-fraud (majority class)
# meh_df = shuffled_df.loc[shuffled_df['target'] == 'meh'].sample(n=1040,random_state=42)
pos_df = shuffled_df.loc[shuffled_df['target'] == 'pos'].sample(n=1628,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([neg_df, pos_df])
normalized_df.head()

,mark,text,dignity,shortcomings,target,lemma_text
534,2,"пульт на вигляд коштує 10 грн. потрібно купувати lg an-mr19ba.ніжки ну дуже здешевлені. пластикові.звук дуже слабенький в плані басу. басу майже нема.споживає 140 !!! ват.здешевлений зовнішньо - весь із чорного пластику, без вставок з іншого матеріалу.55"" занадто великий телек для перегляду т2. кубики видно.на lg 49"" кубиків не видно.",Це LG,Слабенький,neg,"пульта на вигляд коштувати 10 грн . потрібно купувати lg an-mr 19ba . ніжка ну дуже здешевлений . пластиковий . звук дуже слабенький в план бас . бас майже немати . споживати 140 !!! вата . здешевлений зовнішній - весь із чорний пластика , без вставка з інший матеріал . 55 "" занадто великий телека для перегляд т2 . кубик видно . на lg 49 "" кубик не видно ."
7643,1,"користуюсь пів дня) 1.непрацює плей маркет. ніякими маніпуляціями не зміг заставити працювати.2. постійно вилітає браузер.3.оновлення системи не бачить підключення до вайфай і відповідно нічого не знаходить. 4. дуже сильно гріється при перегляді відео а також у браузері. відео не дуже потрібно, а от браузер... при цьому батареї хватає на 3години. якщо не зможу заставити працювати гугл сервіси, буду повертати. не вартий тої ціни, що за нього просять...","Швидкодія. Хороший дисплей, підсвітка та і на вигляд гарний. Браузер, вайфай, блютуз, 3.5мм",Непрацюють гугл сервіси. Гріється і дуже швидко сідає батарея. Сира прошивка. Вилітає браузер.,neg,"користуватися пів день ) 1 . непрацювати плей маркет . ніякий маніпуляція не змогти заставити працювати . 2 . постійно вилітати браузер . 3 . оновлення система не бачити підключення до вайфай і відповідно ніщо не знаходити . 4 . дуже сильно грітися при перегляд відео а також у браузер . відео не дуже потрібно , а от браузер ... при це батарея хватати на 3години . якщо не змогти заставити працювати Гугл сервіс , бути повертати . не вартий той ціна , що за він просити ..."
8945,2,"придбав цей телефон на заміну втопленій nokia7+ (далі n7+). основною причиною, чому обрав данний телефон, є наявність nfc з google pay. через пару днів використання зрозумів чому він настільки дешевий. в метро не можливо подзвонити/прийняти виклик, навіть знахдячись в підземному переході, коли на n7+ я звонив по телеграму з метро в районі м васильківська - голосіївська.датчик наближення раз на 5-7 викликів відпрацьовує некорректно.датчик освітлення живе своїм життям: відпрацьовує корректно лише тоді коли направиш телефон на джерело світла, сама зміна яскравості єкрану відбувається різко, ніякого плавного переходу. тож в тебе або напівтускле зображення або воно дуже яскраве. кнопка google assistant постійно натискається, так як розташована навпроти кнопки ввімкнення, і ти постійно випадково натискаєш її (сам асистент вимкнений).тачскрін підтуплює(до n7+ був meizu екран в якому краще працює ніж цей) коли здавав n7+ в сервісний центр там сказали що на всіх nokia дешевий тачскрін.корпус дуже слизький, без чохла не обійтись. також без чохла та через відсутність виступів на задній панелі навпомацки важко зрозуміти в якому положенні знаходиться телефон: екраном догори чи динамік знизу.логотипи android one та сенсорні кнопки відрізняються від n7+ не дивлячись на те що бренд та ос одні і ті самі, чому так - незрозуміло.завдяки тому що тут встановлений енергоефективний процесор та hd екран вдалося досягти дуже гарної автономності під кінець дня в мене залишається 65-70% заряду при 60-65% у n7+.наявність індикатора сповіщень в порівнянні з n7+(де він відсутній) це однозначно +.лоток на дві сімкарти + катрка пам'яті також можна віднести до переваг, хоч і користуюсь однією sim картою.вибирати цей телефон в якості основного я би на радив, як запасний, згодиться але не більше.","Ціна, NFC, Google Pay, чистний Android, індикатор сповіщень, лоток 2 SIM + microSD","Дешевий тачскрін, слабкий телефонний модуль, датчик наближення, датчик освітлення, слизький корпус",neg,"придбати цей телефон на заміна втоплений nokia7 + ( далі n 7 + ) . основний причина , чому 

In [28]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df['lemma_text'], normalized_df['target'], test_size=0.3, random_state=42)

In [29]:
vectorizer = CountVectorizer(ngram_range = (1,2))
vec = vectorizer.fit(X_train.append(X_test))
print("Total number of features: ", len(vec.get_feature_names()))

Total number of features:  68323


In [30]:
train_features_vectorized = vec.transform(X_train)
test_features_vectorized = vec.transform(X_test)

In [31]:
clf = MultinomialNB()
clf.fit(train_features_vectorized, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
#on bigrams word
predicted = clf.predict(test_features_vectorized)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

         neg       0.82      0.81      0.82       511
         pos       0.79      0.81      0.80       466

    accuracy                           0.81       977
   macro avg       0.81      0.81      0.81       977
weighted avg       0.81      0.81      0.81       977



In [33]:
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial", max_iter=1000,
                         verbose=1)
scores = cross_val_score(lrc, train_features_vectorized, y_train, cv=5, scoring='f1_macro')
scores.mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 731 epochs took 2 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 768 epochs took 1 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

convergence after 777 epochs took 2 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 671 epochs took 1 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s finished


0.8084725013406301

# Висновки

1. Найважче було виділити з датасету нейтральні коментарі. Пробувала різні комбінації, але на нейтральному класі точність була дуже погана, тож вирішила його виключити взагалі. 
2. Пробувала різні біграми - на рівні слів і на рівні символів. Хотілося б розуміти де що краще використовувати, а не підбирати все навмання
3. Фільтрація стоп слів не дала результатів
4. Покращення рішення бачу за рахунок 
    - опрацювання заперечень
    - використання кращої техніки для роботи з незбалансованими класами, ніж undersampling
    - генерація фічей
5. Ну і сильно підкачує часті невідповідність оцінки змісту повідомлення - C'est la vie
6. Моя категорія - "Смартфони, ТВ і електроніка". За рахунок цього вдалося зібрати багато коментарів українською ~ 10 тисяч, але товари там дуже різноманітні і це повпливало на якість класифікатора в свою чергу 